In [239]:
using LinearAlgebra
#Rayleigh Ritz

In [240]:
A = [2 0 0 ; 0 2 1 ; 0 1 2]

3×3 Matrix{Int64}:
 2  0  0
 0  2  1
 0  1  2

In [241]:
eigvals(A)

3-element Vector{Float64}:
 1.0
 2.0
 3.0

In [242]:
eigvecs(A)

3×3 Matrix{Float64}:
  0.0       1.0  0.0
  0.707107  0.0  0.707107
 -0.707107  0.0  0.707107

In [243]:
function gs(M)
    output = zeros(size(M))
    output[:,1] = M[:,1]/norm(M[:,1])
    sumv = 0
    for i = 2 : size(M,1)
        sum = zeros(size(M[:,1],1))
        vi = M[:,i]
        for q = 1 : i
            sum += dot(vi,output[:,q])*output[:,q]
        end
        output[:,i] = (vi - sum)/norm(vi-sum)
    end
    return(output)
end

gs (generic function with 1 method)

In [244]:
V = gs(A)

3×3 Matrix{Float64}:
 1.0  0.0        0.0
 0.0  0.894427  -0.447214
 0.0  0.447214   0.894427

In [245]:
vv = V'*A*V
val = eigvals(vv)
vecs = eigvecs(vv)

3×3 Matrix{Float64}:
  0.0       1.0   0.0
  0.316228  0.0  -0.948683
 -0.948683  0.0  -0.316228

In [246]:
vv

3×3 Matrix{Float64}:
 2.0  0.0  0.0
 0.0  2.8  0.6
 0.0  0.6  1.2

In [247]:
val

3-element Vector{Float64}:
 1.0
 2.0
 3.0

In [248]:
x = V*vecs

3×3 Matrix{Float64}:
  0.0       1.0   0.0
  0.707107  0.0  -0.707107
 -0.707107  0.0  -0.707107

In [249]:
norm(A*x[:,1]-val[1]*x[:,1])

0.0

In [305]:
function arnoldi_iteration(A,b,n)
    #Crée l'espace de Krylov de taille n+1 (b, Ab, ..., A^nb)
    #Renvoie : Q la base hortonormale
    #h la matrice A dans cette base
    m = size(A,1)
    h = zeros(n+1,n)
    Q = zeros(m,n+1)
    q = b/norm(b)
    Q[:,1] = q
    
    for k=1:n
        v = A*q
        for j=1:k+1
            h[j,k] = dot(conj(Q[:,j]),v)
            v = v - h[j,k] * Q[:,j]
        end
        h[k+1,k] = norm(v)
        eps = 1e-12
        if h[k+1,k] > eps
            q = v / h[k+1,k]
            Q[:,k+1] = q
        else
            return Q,h
        end
    end
    return Q,h
end
Q,h = arnoldi_iteration(A,rand([1,50],size(A,1)),size(A,1))

([0.019992004796802236 -0.0007998400478816579 -0.9997998199640288 0.0; 0.9996002398401118 -0.019980004396083467 0.020003997998428277 0.0; 0.019992004796802236 0.9998000598520838 -0.0004000799599701817 0.0], [2.039968025579536 0.9990009392765756 -1.0498270005057653e-13; 0.9990009392765744 1.9600479808179012 0.02000799199546945; 0.0 0.020007991995520193 1.999983993602563; 0.0 0.0 1.0335577617271202e-13])

In [306]:
Q

3×4 Matrix{Float64}:
 0.019992  -0.00079984  -0.9998      0.0
 0.9996    -0.01998      0.020004    0.0
 0.019992   0.9998      -0.00040008  0.0

In [307]:
h

4×3 Matrix{Float64}:
 2.03997   0.999001  -1.04983e-13
 0.999001  1.96005    0.020008
 0.0       0.020008   1.99998
 0.0       0.0        1.03356e-13

In [308]:
hm = h[1:size(h,2),:]

3×3 Matrix{Float64}:
 2.03997   0.999001  -1.04983e-13
 0.999001  1.96005    0.020008
 0.0       0.020008   1.99998

In [309]:
function computeTresholdZero(M)
    lines,column = size(M)
    tol = 1e-12
    for i = 1:lines
        for j = 1:column
            if(M[i,j]<=tol)
                M[i,j] = 0
            end
        end
    end
    return M
end
hm = computeTresholdZero(hm)

3×3 Matrix{Float64}:
 2.03997   0.999001  0.0
 0.999001  1.96005   0.020008
 0.0       0.020008  1.99998

In [310]:
eigvals(hm)

3-element Vector{Float64}:
 1.0000000000000004
 1.9999999999999998
 3.0

In [311]:
q, r = qr(hm)

LinearAlgebra.QRCompactWY{Float64, Matrix{Float64}}
Q factor:
3×3 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}}:
 -0.898092   0.439758    0.00666094
 -0.439808  -0.897989   -0.0136017
  0.0       -0.0151451   0.999885
R factor:
3×3 Matrix{Float64}:
 -2.27145  -1.75924  -0.00879968
  0.0      -1.32109  -0.0482569
  0.0       0.0       1.99948

In [341]:
function checkConv(C)
    tol = 1e-12
    converge = false
    convergence = tr(C) - size(C,1)
    if( convergence <= tol)
        converge = true
    end
    return converge
end

checkConv (generic function with 1 method)

In [342]:
function qrConv(h)
    q1,r1 = qr(h)
    A1 = r1*q1
    q2,r2 = qr(A1)
    A2 = r2*q2
    C = A2/A1    
    while(!checkConv(C))
        A1 = deepcopy(A2)
        q2,r2 = qr(A2)
        A2 = r2*q2
        C = A2/A1
    end
    return q2,r2
end

qrConv (generic function with 1 method)

In [343]:
q,r = qrConv(hm)

([-0.9999999999999869 1.6268951801407436e-7 -4.1998209129051256e-14; -1.6268951801407976e-7 -0.9999999999999534 2.5814944712919987e-7; 0.0 2.581494471292033e-7 0.9999999999999667], [-2.999999999999803 -8.134475549400279e-7 8.243437471977943e-14; 0.0 -1.9999999999999325 7.744483049402881e-7; 0.0 0.0 1.0000000000001013])

In [344]:
q

3×3 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}}:
 -1.0         1.6269e-7   -4.19982e-14
 -1.6269e-7  -1.0          2.58149e-7
  0.0         2.58149e-7   1.0

In [345]:
r

3×3 Matrix{Float64}:
 -3.0  -8.13448e-7  8.24344e-14
  0.0  -2.0         7.74448e-7
  0.0   0.0         1.0

In [346]:
function lanczos(A,b,n)
    m = size(A,1)
    q0 = zeros(m)
    beta = zeros(m+1)
    alpha = zeros(m+1)
    beta[1] = norm(b)
    for j = 1:m
        q1 = b/beta[j]
        u = A*q1-beta[j]*q0
        alpha[j] = u'*q1
        b = u-alpha[j]*q1
        beta[j+1] = norm(b)
        q0 = deepcopy(q1)
    end
    return q1,alpha,beta
        
end
q,alpha,beta = lanczos(A,rand([1,50],size(A,1)),size(A,1))

([-0.9971844109685981 0.0721204632301087 -0.020539944125635697; -0.07498833590105664 -0.9590478409840941 0.2731373064642357; 0.0 0.27390852028957047 0.9617557499244691], [2.039968025579536, 1.9600479808179012, 1.9999839936025627, 0.0], [50.0199960015992, 0.9990009392765744, 0.020007991995520193, 9.396988593091173e-15])

In [347]:
q

3×3 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}}:
 -0.997184    0.0721205  -0.0205399
 -0.0749883  -0.959048    0.273137
  0.0         0.273909    0.961756

In [358]:
computeTresholdZero(q'q)

3×3 Matrix{Float64}:
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0

In [348]:
alpha

4-element Vector{Float64}:
 2.039968025579536
 1.9600479808179012
 1.9999839936025627
 0.0

In [349]:
beta

4-element Vector{Float64}:
 50.0199960015992
  0.9990009392765744
  0.020007991995520193
  9.396988593091173e-15

In [350]:
beta = [i>1e-12 ? i : 0 for i in beta]

4-element Vector{Real}:
 50.0199960015992
  0.9990009392765744
  0.020007991995520193
  0

In [351]:
H = zeros(length(alpha),length(beta))
for i =1:length(alpha)
    H[i,i] = alpha[i]
    if(i < length(alpha))
        H[i,i+1] = beta[i+1]
        H[i+1,i] = beta[i+1]
    end
end
H = H[1:size(H,1)-1,1:size(H,2)-1]

3×3 Matrix{Float64}:
 2.03997   0.999001  0.0
 0.999001  1.96005   0.020008
 0.0       0.020008  1.99998

In [352]:
hm

3×3 Matrix{Float64}:
 2.03997   0.999001  0.0
 0.999001  1.96005   0.020008
 0.0       0.020008  1.99998

In [353]:
q,r = qrConv(H)

([-0.9999999999999869 1.6268951801407404e-7 -4.1998209129050776e-14; -1.6268951801407944e-7 -0.9999999999999534 2.5814944712919743e-7; 0.0 2.5814944712920087e-7 0.9999999999999667], [-2.999999999999803 -8.134475900416424e-7 8.467094227974713e-14; 0.0 -1.9999999999999323 7.744483410814633e-7; 0.0 0.0 1.0000000000001008])

In [354]:
r

3×3 Matrix{Float64}:
 -3.0  -8.13448e-7  8.46709e-14
  0.0  -2.0         7.74448e-7
  0.0   0.0         1.0

In [355]:
q,r = qrConv(hm)

([-0.9999999999999869 1.6268951801407436e-7 -4.1998209129051256e-14; -1.6268951801407976e-7 -0.9999999999999534 2.5814944712919987e-7; 0.0 2.581494471292033e-7 0.9999999999999667], [-2.999999999999803 -8.134475549400279e-7 8.243437471977943e-14; 0.0 -1.9999999999999325 7.744483049402881e-7; 0.0 0.0 1.0000000000001013])

In [356]:
r

3×3 Matrix{Float64}:
 -3.0  -8.13448e-7  8.24344e-14
  0.0  -2.0         7.74448e-7
  0.0   0.0         1.0